In [1]:
# ==========================================
# Day 08 (Task 8): Catalog + Schema + Tables + Permissions + View
# Colab Version (Databricks UC Simulation using DuckDB)
# ==========================================

!pip -q install duckdb pandas

import duckdb
import pandas as pd
import os

con = duckdb.connect(database="task8_uc_simulation.duckdb")

print("Connected to DuckDB (UC Simulation)")

# -------------------------
# 1) Create Structure (Catalog + Schemas)
# -------------------------
# DuckDB doesn't have CREATE CATALOG like Databricks UC,
# so we simulate "commerce" using schema naming.

con.execute("CREATE SCHEMA IF NOT EXISTS commerce_bronze;")
con.execute("CREATE SCHEMA IF NOT EXISTS commerce_silver;")
con.execute("CREATE SCHEMA IF NOT EXISTS commerce_gold;")

print("Schemas created: commerce_bronze, commerce_silver, commerce_gold")


# -------------------------
# 2) Register Tables (Bronze/Silver/Gold)
# -------------------------
# In Databricks: USING DELTA LOCATION '/delta/...'
# Here we simulate with actual tables inside DuckDB

# Bronze events table (raw-ish)
con.execute("""
CREATE TABLE IF NOT EXISTS commerce_bronze.events (
  event_id INTEGER,
  product_name VARCHAR,
  revenue DOUBLE,
  purchases INTEGER,
  conversion_rate DOUBLE
);
""")

# Silver events table (cleaned/refined)
con.execute("""
CREATE TABLE IF NOT EXISTS commerce_silver.events (
  event_id INTEGER,
  product_name VARCHAR,
  revenue DOUBLE,
  purchases INTEGER,
  conversion_rate DOUBLE
);
""")

# Gold products table (business-ready)
con.execute("""
CREATE TABLE IF NOT EXISTS commerce_gold.products (
  product_name VARCHAR,
  revenue DOUBLE,
  purchases INTEGER,
  conversion_rate DOUBLE
);
""")

print("Tables created: bronze.events, silver.events, gold.products")


# -------------------------
# Insert Sample Data
# -------------------------
sample_events = pd.DataFrame({
    "event_id": [1, 2, 3, 4, 5, 6],
    "product_name": ["Mouse", "Keyboard", "Mouse", "Monitor", "Laptop", "Keyboard"],
    "revenue": [5000, 7000, 5200, 15000, 65000, 7100],
    "purchases": [12, 8, 11, 15, 20, 9],
    "conversion_rate": [0.12, 0.08, 0.11, 0.15, 0.20, 0.09]
})

# Load into BRONZE
con.execute("DELETE FROM commerce_bronze.events;")
con.register("sample_events", sample_events)
con.execute("INSERT INTO commerce_bronze.events SELECT * FROM sample_events;")

print("Sample data inserted into commerce_bronze.events")


# -------------------------
# 3) Move Bronze -> Silver (Cleaning / Dedup Example)
# -------------------------
con.execute("DELETE FROM commerce_silver.events;")

con.execute("""
INSERT INTO commerce_silver.events
SELECT DISTINCT * FROM commerce_bronze.events;
""")

print("Bronze → Silver completed")


# -------------------------
# 4) Move Silver -> Gold (Aggregation / Business Table)
# -------------------------
con.execute("DELETE FROM commerce_gold.products;")

con.execute("""
INSERT INTO commerce_gold.products
SELECT
  product_name,
  SUM(revenue) AS revenue,
  SUM(purchases) AS purchases,
  ROUND(AVG(conversion_rate), 4) AS conversion_rate
FROM commerce_silver.events
GROUP BY product_name;
""")

print("Silver → Gold completed")


# -------------------------
# 5) Permissions (Simulated)
# -------------------------
# DuckDB doesn't support GRANT like Databricks UC.
# We'll store "permissions metadata" in a table (GitHub-friendly).

con.execute("""
CREATE TABLE IF NOT EXISTS commerce_gold.permissions_log (
  object_name VARCHAR,
  privilege VARCHAR,
  granted_to VARCHAR
);
""")

con.execute("DELETE FROM commerce_gold.permissions_log;")

permissions_data = [
    ("commerce_gold.products", "SELECT", "analysts@company.com"),
    ("commerce_silver (schema)", "ALL PRIVILEGES", "engineers@company.com"),
]

con.executemany("""
INSERT INTO commerce_gold.permissions_log VALUES (?, ?, ?);
""", permissions_data)

print("Permissions simulated and logged in commerce_gold.permissions_log")


# -------------------------
# 6) Controlled View (Top Products)
# -------------------------
con.execute("""
CREATE OR REPLACE VIEW commerce_gold.top_products AS
SELECT
  product_name,
  revenue,
  conversion_rate
FROM commerce_gold.products
WHERE purchases > 10
ORDER BY revenue DESC
LIMIT 100;
""")

print("View created: commerce_gold.top_products")


# -------------------------
# Show Outputs
# -------------------------
print("\n GOLD PRODUCTS TABLE:")
print(con.execute("SELECT * FROM commerce_gold.products ORDER BY revenue DESC;").df())

print("\n CONTROLLED VIEW (top_products):")
print(con.execute("SELECT * FROM commerce_gold.top_products;").df())

print("\n PERMISSIONS LOG:")
print(con.execute("SELECT * FROM commerce_gold.permissions_log;").df())

print("\n TASK 8 COMPLETED SUCCESSFULLY!")

Connected to DuckDB (UC Simulation)
Schemas created: commerce_bronze, commerce_silver, commerce_gold
Tables created: bronze.events, silver.events, gold.products
Sample data inserted into commerce_bronze.events
Bronze → Silver completed
Silver → Gold completed
Permissions simulated and logged in commerce_gold.permissions_log
View created: commerce_gold.top_products

 GOLD PRODUCTS TABLE:
  product_name  revenue  purchases  conversion_rate
0       Laptop  65000.0         20            0.200
1      Monitor  15000.0         15            0.150
2     Keyboard  14100.0         17            0.085
3        Mouse  10200.0         23            0.115

 CONTROLLED VIEW (top_products):
  product_name  revenue  conversion_rate
0       Laptop  65000.0            0.200
1      Monitor  15000.0            0.150
2     Keyboard  14100.0            0.085
3        Mouse  10200.0            0.115

 PERMISSIONS LOG:
                object_name       privilege             granted_to
0    commerce_gold.produc